# CNN HANDWRITTEN LETTER RECOGNITION
##### ***With fixed learning rate***

### **Imports...**

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import random_split
from sklearn.metrics import classification_report

### **GPU/MPS/CPU...**

In [2]:
if torch.cuda.is_available(): 
    device = "cuda"
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = "cpu"
print(
    f"~~~~~~~~~~~~~~~~~ \n"
    f"Using device: {device} \n"
    f"~~~~~~~~~~~~~~~~~")

~~~~~~~~~~~~~~~~~ 
Using device: mps 
~~~~~~~~~~~~~~~~~


### **CNN...**

In [3]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=26): 
       
        super(CNN, self).__init__()

        #1ST CONVOLUTIONAL LAYER
        self.conv1 = nn.Conv2d( 
            in_channels=in_channels,  
            out_channels=64, 
            kernel_size=3, 
            stride=1,  
            padding=1) 
        self.bn1 = nn.BatchNorm2d(64)
        
        #2ND CONVOLUTIONAL LAYER
        self.conv2 = nn.Conv2d(  
            in_channels=64,  
            out_channels=128, 
            kernel_size=3, 
            stride=1, 
            padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        #3RD CONVOLUTIONAL LAYER
        self.conv3 = nn.Conv2d(
            in_channels = 128,
            out_channels = 256,
            kernel_size = 3,
            stride = 1,
            padding = 1)
        self.bn3 = nn.BatchNorm2d(256)
        
        #POOLING LAYER
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        #DROPOUT LAYER (FOR REGULARIZATION)
        self.drop_conv = nn.Dropout2d(p=0.2)
        self.drop_fc = nn.Dropout(p=0.5) 
        
        #FULLY CONNECTED LAYER
        self.adap = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(256, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) 
        x = self.pool(x) 
        x = self.drop_conv(x)          

        x = F.relu(self.bn2(self.conv2(x)))  
        x = self.pool(x)
        x = self.drop_conv(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.drop_conv(x)

        x = self.adap(x)
        x = x.view(x.size(0),-1)

        x = F.relu(self.fc1(x))
        x = self.drop_fc(x)       
        x = self.fc2(x)            

        return x

### **Hyperparamethers...**

In [4]:
num_classes = 26 
learning_rate = 0.005
batch_size = 200
num_epochs = 50
weight_decay = 0

### **Data loading, preprocessing and transforming...**

In [5]:
def emnist_rotation(img):
    img = img.rotate(-90, expand = True)
    img = img.transpose(Image.FLIP_TOP_BOTTOM)
    return img

def train_transform():
    return transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((32,32),antialias=True), 
    transforms.Lambda(emnist_rotation),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1), scale=(0.9,1.1), shear = 3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307],std=[0.3081])])

def test_transform():
    return transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32,32),antialias=True),
    transforms.Lambda(emnist_rotation),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307],std=[0.3081])])

# ~~~~~~~~~~ SPLITTING THE TRAINING SET ~~~~~~~~~~~
train_base = datasets.EMNIST(root='emnist-letters-train', split='letters', train=True, download=True, transform=test_transform(), target_transform=lambda y: y-1)
train_set = int(0.8*len(train_base))
val_set = len(train_base)-train_set
train_dataset, val_dataset = random_split(train_base, [train_set,val_set])

# ~~~~~~~~~~ TRAIN ~~~~~~~~~~~
train_dataset.dataset.transform = train_transform()
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# ~~~~~~~~~~ VALIDATION ~~~~~~~~~~~
val_dataset.dataset.transform = test_transform()
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

# ~~~~~~~~~~ TEST ~~~~~~~~~~~
test_dataset = datasets.EMNIST(root='emnist-letters-test', split='letters', train=False, download=True, transform=test_transform(), target_transform=lambda y: y-1)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### **Initialize network with loss and optimizer...**

In [6]:
model = CNN(in_channels=1, num_classes=num_classes).to(device)

In [7]:
criterion = nn.CrossEntropyLoss(label_smoothing= 0)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay= weight_decay)

### **Train and evaluate...**

In [8]:
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Learning rate: {optimizer.param_groups[0]['lr']:.6f}")

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # TRAINING 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    model.train()
    train_correct, train_total, train_loss = 0, 0, 0.0

    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        scores = model(data) # Forward
        loss = criterion(scores, targets)

        loss.backward() # Backward
        optimizer.step()

        batch_size = targets.size(0)
        train_loss += loss.item() * batch_size
        
        _, preds = scores.max(1)
        train_correct += (preds == targets).sum().item()
        train_total += batch_size
    
    train_loss /= train_total
    train_acc = 100.0 * train_correct / train_total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # EVALUATION 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0.0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            
            loss = criterion(scores, y)
            batch_size = y.size(0)
            val_loss += loss.item() * batch_size

            _, predictions = scores.max(1) 
            val_correct += (predictions == y).sum().item() 
            val_total += batch_size

    val_loss /= val_total
    val_acc = float(val_correct) / float(val_total) * 100.0
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
        
    print(
        f" \n "
        f"|~|TRAINING:|~| Accuracy: {train_acc:.2f}, Loss: {train_loss:.4f}. \n "
        f"|~|EVALUATING:|~| Accuracy: {val_acc:.2f}, Loss: {val_loss:.4f}. \n " 
        f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
        )

    print()

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# FINAL METRICS 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
final_train_loss = train_losses[-1]
final_train_acc = train_accuracies[-1]
final_val_loss = val_losses[-1]
final_val_acc = val_accuracies[-1]

print("===== Training Complete =====")
print(f"Final Train Loss: {final_train_loss:.4f}")
print(f"Final Train Accuracy: {final_train_acc:.2f}%")
print(f"Final Test Loss: {final_val_loss:.4f}")
print(f"Final Test Accuracy: {final_val_acc:.2f}%")
    

Epoch [1/50]
Learning rate: 0.005000


100%|██████████| 500/500 [00:52<00:00,  9.50it/s]


 
 |~|TRAINING:|~| Accuracy: 54.67, Loss: 1.4270. 
 |~|EVALUATING:|~| Accuracy: 86.23, Loss: 0.4391. 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch [2/50]
Learning rate: 0.005000


100%|██████████| 500/500 [00:45<00:00, 10.92it/s]


 
 |~|TRAINING:|~| Accuracy: 81.50, Loss: 0.5821. 
 |~|EVALUATING:|~| Accuracy: 91.09, Loss: 0.2693. 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch [3/50]
Learning rate: 0.005000


...

 
 |~|TRAINING:|~| Accuracy: 94.73, Loss: 0.1453. 
 |~|EVALUATING:|~| Accuracy: 95.03, Loss: 0.1686. 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch [49/50]
Learning rate: 0.005000


100%|██████████| 500/500 [00:47<00:00, 10.59it/s]


 
 |~|TRAINING:|~| Accuracy: 94.89, Loss: 0.1403. 
 |~|EVALUATING:|~| Accuracy: 95.08, Loss: 0.1719. 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch [50/50]
Learning rate: 0.005000


100%|██████████| 500/500 [00:47<00:00, 10.59it/s]


 
 |~|TRAINING:|~| Accuracy: 94.78, Loss: 0.1433. 
 |~|EVALUATING:|~| Accuracy: 94.82, Loss: 0.1704. 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

===== Training Complete =====
Final Train Loss: 0.1433
Final Train Accuracy: 94.78%
Final Test Loss: 0.1704
Final Test Accuracy: 94.82%


In [ ]:
torch.save(model.state_dict(), "my_CNN_fixed.pth")

### **Plotting...**

In [ ]:
def plot_stats(train_acc, val_acc, train_loss, val_loss):
    epochs = range(1, len(train_acc)+1)
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,4))

    best_epoch = val_acc.index(max(val_acc))
    best_acc = val_acc[best_epoch]

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ACCURACY PLOT 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ax1.plot(epochs, train_acc, label="Train Accuracy", color='salmon')
    ax1.plot(epochs, val_acc, label="Val Accuracy", color='sienna')

    ax1.scatter(best_epoch + 1, best_acc, color='indianred', s=50, zorder=5, label=f"Best Test acc ({best_acc:.2f}%) at epoch {best_epoch}")
    ax1.annotate(f'{best_acc:.2f}%',
                 xy=(best_epoch + 1, best_acc),
                 xytext=(best_epoch + 1, best_acc - 6),
                 ha='center', color='indianred',
                 arrowprops=dict(arrowstyle='->', color='black'))
    ax1.set_title('Train vs. val accuracy')
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Accuracy (%)", color='Sienna')
    ax1.tick_params(axis='y', labelcolor='Sienna')
    ax1.legend(loc='lower right')
    ax1.grid(True, linestyle='--', alpha=0.5)

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # LOSS PLOT 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ax2.plot(epochs, train_loss, label="Train Loss", color='salmon')
    ax2.plot(epochs, val_loss, label="Val Loss", color='sienna')
    ax2.set_title('Train vs. val loss')
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Loss", color='sienna')
    ax2.tick_params(axis='y', labelcolor='sienna')
    ax2.legend(loc='upper right')
    ax2.grid(True, linestyle='--', alpha=0.5)

    plt.savefig("Fixed_train_results_plot.png")
    plt.tight_layout()
    plt.show()

plot_stats(
    train_accuracies, 
    val_accuracies, 
    train_losses, 
    val_losses
)

### **Save results in file...**

In [ ]:
results = "Fixed_train_results.txt"
with open(results, "w") as f: 
    f.write("Epoch, Train loss, Train acc, Val loss, Val acc\n")
    for i in range(num_epochs):
        f.write(
            f"{i+1}, "
            f"{train_losses[i]:.4f}, "
            f"{train_accuracies[i]:.2f}%, "
            f"{val_losses[i]:.4f}, "
            f"{val_accuracies[i]:.2f}%\n")
        
    f.write("\n===== Final Metrics =====\n")
    f.write(f"Final Train Loss: {final_train_loss:.4f}\n")
    f.write(f"Final Train Accuracy: {final_train_acc:.2f}%\n")
    f.write(f"Final Val Loss: {final_val_loss:.4f}\n")
    f.write(f"Final Val Accuracy: {final_val_acc:.2f}%\n")

### **Testing the model...**

In [ ]:
def accuracy(model, dataloader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for img, labl in dataloader:
            img, labl = img.to(device), labl.to(device)
            output = model(img)
            _, expected = torch.max(output, 1)
            total += labl.size(0)
            correct += (expected == labl).sum().item()
    accuracy = correct / total 
    return accuracy

def x_eval(model, dataloader, device):
    model.eval()
    all_predictions, all_labls = [], []
    with torch.no_grad():
        for img, labl in dataloader:
            img, labl = img.to(device), labl.to(device)
            output = model(img)
            _, predicted = torch.max(output, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labls.extend(labl.cpu().numpy())
    return classification_report(all_labls, all_predictions, digits=4)

def full_evaluation(model, dataloader, device):
    acc = accuracy(model, dataloader, device)
    class_rep = x_eval(model, dataloader, device)
    result_file = "Fixed_test_result.txt"
    with open(result_file, "w") as f: 
        f.write(
            f"Test Accuracy: {acc} \n"
            f"Classification_report: {class_rep}")
    print(f" Test accuracy is: {acc:.4f}.")
    print(class_rep)

In [ ]:
model.load_state_dict(torch.load("my_CNN_fixed.pth"))
model.to(device)
full_evaluation(model, test_loader, device=device)